In [11]:
subject = 'sample' 
atlas = 'dkt'

In [12]:
import os
os.chdir('..')
from src.mmvt_addon.scripts import run_mmvt
run_mmvt.run(subject, atlas, run_in_background=False)

argv: --subject sample --atlas dkt
*********** sample ***********
/local_mount/space/thibault/1/users/npeled/blender-2.78a-linux-glibc211-x86_64/blender /autofs/space/thibault_001/users/npeled/mmvt/sample_dkt.blend  --python "/local_mount/space/thibault/1/users/npeled/code/mmvt/src/mmvt_addon/scripts/run_mmvt.py" --subject "sample" --subjects "" --atlas "dkt" --real_atlas "aparc.DKTatlas40" --bipolar "False" --debug "False" --blender_fol "/local_mount/space/thibault/1/users/npeled/blender-2.78a-linux-glibc211-x86_64" 
start!
After Blender call
Finish! For more details look in /local_mount/space/thibault/1/users/npeled/code/mmvt/logs/run_mmvt.log


In [13]:
from pizco import Proxy
mmvt = Proxy('tcp://127.0.0.1:8000')

In [14]:
import os.path as op
import mne

mne_sample_data_fol = mne.datasets.sample.data_path()
meg_subject = 'sample'
stc_fname = op.join(mne_sample_data_fol, 'MEG', 'sample', 'sample_audvis-meg-rh.stc')
stc = mne.read_source_estimate(stc_fname)

In [20]:
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Label, Button, Layout, Image

T = stc.lh_data.shape[1] 
play = widgets.Play(value=0, min=0, max=T, step=1, interval=1000)
slider = widgets.IntSlider(value=0, min=0, max=T, step=1)
widgets.jslink((play, 'value'), (slider, 'value'))
quality_slider = widgets.IntSlider(value=40, min=0, max=100, step=1, layout=Layout(width='100%'))
threshold_slider = widgets.FloatSlider(value=2, min=0, max=10, step=0.1, layout=Layout(width='20%'))

In [16]:
import numpy as np
def create_stc_t(stc, t):
    data = np.concatenate([stc.lh_data[:, t:t + 1], stc.rh_data[:, t:t + 1]])
    vertices = [stc.lh_vertno, stc.rh_vertno]
    stc_t = mne.SourceEstimate(data, vertices, stc.tmin + t * stc.tstep, stc.tstep, subject=subject)
    return stc_t

def plot_stc(stc, t):
    # return mmvt.plot_stc(stc, t, threshold=threshold_slider.value,  save_image=True)
    stc_t = create_stc_t(stc, t)
    stc_t_smooth = mne.morph_data(meg_subject, subject, stc_t, n_jobs=n_jobs, grade=vertices_to)
    meg_fname = mmvt.plot_stc_t(stc_t_smooth.rh_data, stc_t_smooth.lh_data, t, threshold=threshold_slider.value, save_image=True)
    return meg_fname, stc_t_smooth
    
n_jobs = 4
t = 0 # ms
vertices_to = mne.grade_to_vertices(subject, None)    
meg_fname, stc_t_smooth = plot_stc(stc, t)
file = open(meg_fname, "rb")
meg_image = Image(value=file.read(), format='png', width=800, height=450)

Morphing data...
    Left-hemisphere map read.
    Right-hemisphere map read.
    Left-hemisphere map read.
    Right-hemisphere map read.
    17 smooth iterations done.


[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:    0.4s finished


    14 smooth iterations done.
[done]


[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:    0.4s finished


In [17]:
def update_image(fname=''):
    fname = mmvt.save_image('stc', index=slider.value) if fname == '' else fname
    file = open(fname, "rb")
    meg_image.value = file.read()

sagital_button = Button(description="Sagital")
sagital_button.on_click(lambda x:(mmvt.show_sagital(), update_image()))
coronal_button = Button(description="Coronal")
coronal_button.on_click(lambda x:(mmvt.show_coronal(), update_image()))
axial_button = Button(description="Axial")
axial_button.on_click(lambda x:(mmvt.show_axial(), update_image()))
render_button = Button(description='Render')
update_button = Button(description='Update')
update_button.on_click(lambda x:(update_image()))
render_button.on_click(lambda x:(mmvt.hide_subcorticals(), mmvt.set_brain_transparency(1), mmvt.set_to_camera_view(),
                                 update_image(mmvt.render_image(quality=quality_slider.value))))

In [21]:
def on_value_change(change):
    t = change['new']
    meg_fname, stc_t_smooth = plot_stc(stc, t)
    file = open(meg_fname, "rb")
    meg_image.value = file.read()

slider.observe(on_value_change, names='value') 
VBox([HBox([meg_image, VBox([sagital_button, coronal_button, axial_button, update_button,
                             Label('Quality:'), quality_slider, render_button])]),
     HBox([play, slider, Label('Threshold:'), threshold_slider])])

In [22]:
mmvt.rotate_brain(0, 0, 0.02, True)

True

b"Switching to fully guarded memory allocator.\nBlender 2.78 (sub 0)\nBuild: 2016-10-24 15:02:41 Linux Release\nargv[0] = /local_mount/space/thibault/1/users/npeled/blender-2.78a-linux-glibc211-x86_64/blender\nargv[1] = /autofs/space/thibault_001/users/npeled/mmvt/sample_dkt.blend\nargv[2] = --python\nargv[3] = /local_mount/space/thibault/1/users/npeled/code/mmvt/src/mmvt_addon/scripts/run_mmvt.py\nargv[4] = --subject\nargv[5] = sample\nargv[6] = --subjects\nargv[7] = \nargv[8] = --atlas\nargv[9] = dkt\nargv[10] = --real_atlas\nargv[11] = aparc.DKTatlas40\nargv[12] = --bipolar\nargv[13] = False\nargv[14] = --debug\nargv[15] = False\nargv[16] = --blender_fol\nargv[17] = /local_mount/space/thibault/1/users/npeled/blender-2.78a-linux-glibc211-x86_64\nread file /homes/5/npeled/.config/blender/2.78/config/startup.blend\n  Version 275 sub 0 date 2015-07-07 14:56 hash c27589e\n\nordered\n OBCube\n OBLamp\n OBCamera\nread file /homes/5/npeled/.config/blender/2.78/config/userpref.blend\n  Versi